In [1]:
import pandas as pd

In [2]:
file = "rawdata.csv" # define the csv file name

In [3]:
# read the csv
data = pd.read_csv(
    file,
    usecols=["LATITUDE", "LONGITUDE"]
)

In [4]:
data.head()

,LATITUDE,LONGITUDE
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,40.667202,-73.866500
4,40.683304,-73.917274


In [5]:
data.dropna().head()

,LATITUDE,LONGITUDE
3,40.667202,-73.866500
4,40.683304,-73.917274
6,40.709183,-73.956825
7,40.868160,-73.831480
8,40.671720,-73.897100


In [6]:
# drop rows with no coordinates
data = data.dropna()

# drop rows with 0 in all cols
data = data.loc[(data != 0).any(1)]

# data overview
data.describe()

C:\Users\LamKingYiu\AppData\Local\Temp\ipykernel_15748\586527192.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  data = data.loc[(data != 0).any(1)]


,LATITUDE,LONGITUDE
count,1.798167e+06,1.798167e+06
mean,4.072416e+01,-7.392729e+01
std,7.958958e-02,9.896603e-01
min,3.078418e+01,-2.013600e+02
25%,4.066835e+01,-7.397495e+01
50%,4.072122e+01,-7.392752e+01
75%,4.076974e+01,-7.386722e+01
max,4.334444e+01,-3.276851e+01


In [7]:
# change to numpy for sklearn
X = data.to_numpy()

In [8]:
# select sub-samples if needed
# subX = X[::20]

# choose the full dataset
subX = X

# print out dataset size
subX.shape

(1798167, 2)

In [ ]:
## memory error due DBSCAN complexity O(n^2)
## DO NOT RUN

import numpy as np
import time
from sklearn import metrics
from sklearn.cluster import DBSCAN

start_time = time.time()
db = DBSCAN(
    eps=0.1, 
    min_samples=100, 
    n_jobs=-1, # use 4 cores
    algorithm='ball_tree', 
    metric='haversine'
).fit(np.radians(subX))
labels = db.labels_
end_time = time.time()

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Running time: ", end_time - start_time)
print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

In [18]:
# USE OPTICS to perform DBSCAN

import numpy as np
import time
from sklearn import metrics
from sklearn.cluster import OPTICS

start_time = time.time()
op = OPTICS(
    max_eps=0.1/6371., 
    min_samples=1000,
    # eps setting see:
    # https://stackoverflow.com/questions/34579213/dbscan-for-clustering-of-geographic-location-data
    eps=0.05/6371., # 0.05 km / 50m
    cluster_method='dbscan',
    n_jobs=-1,
    algorithm='ball_tree', 
    metric='haversine'
).fit(np.radians(subX)) # convert GPS coordinates to radians
labels = op.labels_
end_time = time.time()

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Running time: ", end_time - start_time)
print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

Running time:  15817.286792516708
Estimated number of clusters: 13
Estimated number of noise points: 1784554


In [19]:
# label of all points, starting with 0
# -1 if belongs to noise
labels = np.array(op.labels_)

In [20]:
coordinates = subX[labels != -1]

In [21]:
clusters = labels[labels != -1]

In [22]:
np.unique(clusters)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [23]:
#number of centres
no_centre = np.unique(labels[labels!=-1]).shape[0]

# create a np array to calculate the centre
centres=np.zeros(shape=(no_centre,2))

for i in range(no_centre):
    centres[i] = np.average(subX[labels==i],axis=0)

In [24]:
# list of all centres
centres.tofile("centres")

# list of all coordinates
coordinates.tofile("coordinates")

# list of labels
clusters.tofile("labels")

In [25]:
# output coordinates to csv
pd.DataFrame(
    data = coordinates,
    columns = ("LATITUDE", "LONGITUDE")
).to_csv("coordinates.csv")

In [26]:
# output centres to csv
pd.DataFrame(
    data = centres,
    columns = ("LATITUDE", "LONGITUDE")
).to_csv("centres.csv")

In [27]:
# to read the file
# use np.fromfile()

In [28]:
centres.shape

(13, 2)

In [29]:
coordinates.shape

(13613, 2)

In [30]:
clusters.shape

(13613,)

In [31]:
coordinates[clusters == 0]

array([[ 40.861862, -73.91275 ],
       [ 40.86173 , -73.91182 ],
       [ 40.861862, -73.91282 ],
       ...,
       [ 40.861862, -73.91275 ],
       [ 40.86173 , -73.91182 ],
       [ 40.861862, -73.91275 ]])